# CNN

![CNN-example](./images/08-CNN-example.png)

initial image is convoluted with filter and the filter moves slides incrementally over the whole image

![max-pooling](./images/09-maxpooling-example.png)


![](./images/10-cnn-model-diagram.png)

In [3]:
# import package
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [4]:
# device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [27]:
# hyper-parameters
num_epochs = 4
batch_size = 4
learning_rate = 0.01

In [28]:
# dataset as PIL Image images of range [0,1]
# we tranform them to Tensors of normalized range [-1,1]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [29]:
# create dataset and dataloader
train_dataset = torchvision.datasets.CIFAR10(root='./datasets', 
                                             train=True, 
                                             download=True, 
                                             transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./datasets', 
                                             train=False, 
                                             download=True, 
                                             transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

Files already downloaded and verified
Files already downloaded and verified


![](./images/11-conv-length.png)

In [30]:
# implement conv net
class ConvNet(nn.Module):
    def __init__(self):
        # input channel size, output channel size, kernel size
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        out = self.pool(F.relu(self.conv1(x)))
        out = self.pool(F.relu(self.conv2(out)))
        out = out.reshape(-1,16*5*5)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [31]:
model = ConvNet().to(device)

In [32]:
# loos
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr =learning_rate)

In [33]:
n_total_steps = len(train_loader)

In [34]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape = [4,3,32,32] = 4 3 1024
        # input_layer = 3 input channel, 6 output channel, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%2000 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss {loss.item():.4f}')

print("finish training")

epoch 1/4, step 2000/12500, loss 1.7137
epoch 1/4, step 4000/12500, loss 0.6308
epoch 1/4, step 6000/12500, loss 1.7495
epoch 1/4, step 8000/12500, loss 2.1425
epoch 1/4, step 10000/12500, loss 2.9309
epoch 1/4, step 12000/12500, loss 2.1923
epoch 2/4, step 2000/12500, loss 1.5516
epoch 2/4, step 4000/12500, loss 1.3213
epoch 2/4, step 6000/12500, loss 3.2398
epoch 2/4, step 8000/12500, loss 1.1960
epoch 2/4, step 10000/12500, loss 1.3733
epoch 2/4, step 12000/12500, loss 0.7607
epoch 3/4, step 2000/12500, loss 1.5053
epoch 3/4, step 4000/12500, loss 3.8853
epoch 3/4, step 6000/12500, loss 0.8180
epoch 3/4, step 8000/12500, loss 1.3126
epoch 3/4, step 10000/12500, loss 1.4498
epoch 3/4, step 12000/12500, loss 1.1900
epoch 4/4, step 2000/12500, loss 0.8247
epoch 4/4, step 4000/12500, loss 1.1217
epoch 4/4, step 6000/12500, loss 0.8251
epoch 4/4, step 8000/12500, loss 0.8490
epoch 4/4, step 10000/12500, loss 1.2841
epoch 4/4, step 12000/12500, loss 1.5312
finish training


In [35]:
# test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        output = model(images)

        # value, index
        _, predictions = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predictions == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    
    print(f'accuracy = {acc}%')

accuracy = 10.02%
